## Read file

In [1]:
import pandas as pd
import numpy as np

In [2]:
file = open('articles.txt', 'r', encoding = 'utf8')
columns = ["pubId", "is_hourly", "seqId", "on_homepage", "canonicalUrl",
                   "firstScrape", "lang_iso", "lang_reliability", "title", "text"]
articles_dt = file.read().split('\n')[:-1]

In [3]:
pubId, is_hourly, seqId, on_homepage, canonicalUrl,firstScrape,lang_iso, lang_reliability, title,text = [],[],[],[],[], [],[],[],[],[]
for article in articles_dt:    
    row = article.split('\t')
    pubId.append(row[0])
    is_hourly.append(row[1])
    seqId.append(row[2])
    on_homepage.append(row[3])
    canonicalUrl.append(row[4])
    firstScrape.append(row[5])
    lang_iso.append(row[6])
    lang_reliability.append(row[7])
    title.append(row[8])
    text.append(row[9])
articles_df = pd.DataFrame()

In [4]:
articles_df['pubId'], articles_df['is_hourly'], articles_df['seqId'], articles_df['on_homepage'], articles_df['canonicalUrl'], articles_df['firstScrape'], articles_df['lang_iso'], articles_df['lang_reliability'],articles_df['title'], articles_df['text'] = pubId, is_hourly, seqId, on_homepage, canonicalUrl,firstScrape,lang_iso, lang_reliability, title,text


In [5]:
print(articles_df.shape)
articles_df['text'][0]

(213605, 10)


'by Phoenix Capita… - Jul 19, 2018 8:49 am ### The Government is planning a $1 trillion deficit next year. ### Jul 19, 2018 8:49 AM ### Financial Terrorism In The UK – Collusion between Government, Regulators & Two Bailed-Out UK Banks ### by GoldCore - Jul 19, 2018 8:01 am ### Neil Mitchell has waged a long battle for justice for UK victims of the UK taxpayer owned bank RBS and HBOS ### Jul 19, 2018 8:01 AM ### 0 ### SHARES ### In a move that many had expected, President Trump lashed out over Twitter in response to Brussels\' decision to slap Google with an record €4.3bn ($5 billion) antitrust fine, exclaiming "I told you so!" before suggesting that the move was further evidence that the EU had "taken advantage of the U.S., but not for long!" ### I told you so! The European Union just slapped a Five Billion Dollar fine on one of our great companies, Google. They truly have taken advantage of the U.S., but not for long! ### — Donald J. Trump (@realDonaldTrump) July 19, 2018 ### On Wedne

## Clean data

In [6]:
#exclude the lang_reliability = 0 which the detected language is not reliable
articles_df = articles_df[articles_df['lang_reliability'] == '1']
articles_df.shape

(211577, 10)

In [7]:
## text normzalization
from nltk.corpus import stopwords
import re

STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a paragrapy
        
        return: modified initial string
    """
    text = re.sub(r"[^\w\s]", '', text) ## Remove all non-word characters (everything except numbers and letters)
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text
    
articles_df['text'] = articles_df['text'].apply(clean_text)

In [8]:
articles_df['text'][0]

'phoenix capita jul 19 2018 849 government planning 1 trillion deficit next year jul 19 2018 849 financial terrorism uk collusion government regulators two bailedout uk banks goldcore jul 19 2018 801 neil mitchell waged long battle justice uk victims uk taxpayer owned bank rbs hbos jul 19 2018 801 0 shares move many expected president trump lashed twitter response brussels decision slap google record 43bn 5 billion antitrust fine exclaiming told suggesting move evidence eu taken advantage us long told european union slapped five billion dollar fine one great companies google truly taken advantage us long donald j trump realdonaldtrump july 19 2018 wednesday longawaited move google parent alphabet slapped record fine ordered change way installs search web browser apps onto android mobile devices company given midoctober stop illegal practices eu phrased contracts handset manufacturers failure comply result 5 fine revenue penalty equal amount netherlands kicks eu budget year higher punis

In [9]:
articles_df['title'] = articles_df['title'].apply(clean_text)

In [10]:
articles_df['title'][0]

'told trump hits eu 5 billion google fine zero hedge'

## bag of word

### 1. uni-gram

In [22]:
comb = []
for i in articles_df.index:
    comb.append(articles_df['title'][i] + ' ' + articles_df['text'][i])

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

cv = CountVectorizer()
text_counts= cv.fit_transform(comb)

In [25]:
text_counts.shape

(211577, 825770)

In [32]:
np.sum(text_counts[0,:].todense())

221

In [33]:
## save result
np.save('BagOfWord_output.npy', text_counts)    # .npy extension is added if not given

In [34]:
# load saved result
#test = np.load('BagOfWord_output.npy', allow_pickle=True)
#np.asmatrix(test)[0,0]

### 2. bi-gram

In [35]:
cv2 = CountVectorizer(ngram_range=(2, 2))
text_counts2= cv2.fit_transform(comb)

In [36]:
text_counts2.shape

(211577, 13000246)

In [37]:
np.save('BagOfWord2_output.npy', text_counts)  

## word2vec

In [38]:
from gensim.models import word2vec,KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


In [121]:
#comb.index = range(0,len(comb))

'told trump hits eu 5 billion google fine zero hedge'

In [39]:
def word2vec(text): 
    article_vect = np.empty([len(text), 300])
    for i in range(0,len(text)):
        word_list = text[i].split(' ')
        for word in word_list:
            total_vec = np.zeros(300)
            if word in model:
                vec = model[word]
                total_vec = total_vec + vec
        article_vect[i,:] = total_vec/len(word_list)
    return article_vect

In [40]:
article_vect = word2vec(comb)

In [41]:
article_vect.shape

(211577, 300)